# control estimation of attraction in the presence of diffusion
Tim Tyree<br>
11.2.2021

In [1]:
import cupy as cp, numpy as np
import time
import matplotlib.pyplot as plt
from lib import *
from scipy.signal import savgol_filter
import dask.bag as db

In [2]:
darkmode=True
if darkmode:
    # For darkmode plots
    from jupyterthemes import jtplot
    jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

In [3]:
seed=42
# #random number stream A
# rs = RandomState(MT19937(SeedSequence(seed)))
# #random number stream B
# # rs = RandomState(MT19937(SeedSequence(987654321)))
np.random.seed(seed)
cp.random.seed(seed)

# define module

In [61]:
def gener_positions_control(
        num_pairs=1500, #number of independent pairs of particles
        a=5, #cm^2/s known attraction coefficient 
        D=1, #cm^2/s known diffusion coefficient
        Dt=1e-5,
        print_every=100000,
        rend=0.001, #cm
        printing=False
    ):
    '''
    Example Usage:
    positions_out,mean_array,std_array=gener_positions_control(a=a,D=D,printing=True)
    '''
    #initialize particles locations as being at the same location for
    t=0
    x1=0
    x2=0.5
    y1=0
    y2=0.5
    zero_values=np.zeros(num_pairs)

    x1_values=zero_values.copy()+x1
    x2_values=zero_values.copy()+x2
    y1_values=zero_values.copy()+y1
    y2_values=zero_values.copy()+y2

    stepscale = np.sqrt(2 * D * Dt)
    impulse_prefactor = a * Dt

    tmax=10#0.1#1 #seconds
    num_steps=np.int(np.around(tmax/Dt))

    position_array=np.zeros(shape=(num_steps,4,num_pairs))+np.nan
    mean_array=np.zeros(shape=(num_steps,3))
    std_array=np.zeros(shape=(num_steps,3))
    num_steps

    #GOAL: add support for still_running
    
    boo_still_running=zero_values==zero_values
    #TODO: initiallize still_running to True
    #TODO: compute boo where particles become smaller than R_

    if printing:
        print(f"running simulation for {num_steps} steps...")
    start=time.time()
    for step_num in range(num_steps):
        #compute the distance between each pair of particles
        dx_values=(x2_values-x1_values)
        dy_values=(y2_values-y1_values)
        Rsq_values=dx_values*dx_values+dy_values*dy_values

        #compute the attractive step between all pairs
        f_values=impulse_prefactor/Rsq_values
        F1x=f_values*dx_values
        F1y=f_values*dy_values

        #compute the diffusive step between all pairs
        dxW1_values=stepscale*np.random.normal(size=num_pairs)
        dxW2_values=stepscale*np.random.normal(size=num_pairs)
        dyW1_values=stepscale*np.random.normal(size=num_pairs)
        dyW2_values=stepscale*np.random.normal(size=num_pairs)

        #the mean distance between all pairs of particles
        R_values=np.sqrt(Rsq_values)
        mean_R=np.mean(R_values)
        std_R=np.mean(R_values)

        #identify any particles that didn't get within the threshold distance
        boo_still_running=R_values>rend

        #the mean magnitude of the diffusive step
        diffusive_step_values=0.5*np.sqrt(dxW1_values**2+dyW1_values**2)+0.5*np.sqrt(dxW2_values**2+dyW2_values**2)
        mean_diffusive_step=np.mean(diffusive_step_values)
        std_diffusive_step=np.std(diffusive_step_values)

        #the mean magnitude of the attractive step
        attractive_step_values=np.sqrt(F1x**2+F1y**2)
        mean_attractive_step=np.mean(attractive_step_values)
        std_attractive_step=np.std(attractive_step_values)
        if printing:
            if (step_num+1) % print_every == 0:
                relative_percent=100*mean_diffusive_step/mean_attractive_step
                print(f"Simulation {100*(step_num+1)/num_steps:.0f}% completed: R={mean_R:.4f}+/-{1.96*std_R:.4f}, diffusion/attraction is {relative_percent:.2f}%")
                # print(f"the mean range is {mean_R:.4f} cm")
                # print(f"the mean diffusive step was {100*mean_diffusive_step/mean_attractive_step:.2f}% larger than the mean attractive step")

        #compute the net change in position 
        x1step_values=F1x+dxW1_values
        y1step_values=F1y+dyW1_values
        x2step_values=-F1x+dxW2_values
        y2step_values=-F1y+dyW2_values

        #update particle locations if they are still running
        x1_values[boo_still_running]=x1_values[boo_still_running]+x1step_values[boo_still_running]
        y1_values[boo_still_running]=y1_values[boo_still_running]+y1step_values[boo_still_running]
        x2_values[boo_still_running]=x2_values[boo_still_running]+x2step_values[boo_still_running]
        y2_values[boo_still_running]=y2_values[boo_still_running]+y2step_values[boo_still_running]
        #DONE: verified the mean distance between particles got smaller    
        # if (step_count+1) % save_every == 0:
        #save particle locations to a numpy array with the correct number of positions
        # x1,y1,x2,y2
        position_array[step_num,0,boo_still_running]=x1_values[boo_still_running]
        position_array[step_num,1,boo_still_running]=y1_values[boo_still_running]
        position_array[step_num,2,boo_still_running]=x2_values[boo_still_running]
        position_array[step_num,3,boo_still_running]=y2_values[boo_still_running]
        mean_array[step_num,0]=mean_R
        mean_array[step_num,1]=mean_diffusive_step
        mean_array[step_num,2]=mean_attractive_step
        #save results for quick plotting
        std_array[step_num,0]=std_R
        std_array[step_num,1]=std_diffusive_step
        std_array[step_num,2]=std_attractive_step

    
    if printing:
        print(f"simulation complete!\nTotal run time: {time.time()-start:.4f} seconds")
        print(f"the number of particles that didn't finished is {sum(boo_still_running)} out of {boo_still_running.shape[0]}.")

    #GOAL: vectorized processing
    # DONE: plot the basic results versus time and verify that they look reasonable
    # DONE(optional): compute the moving average of particle locations
    # DONE: compute all ranges
    # DONE: find the row index where the positions first become closer than R_thresh=0.001 cm

    #     ylabel_lst=['R (cm)','diffusive\nstep size (cm)','attractive\nstep size (cm)']
    #     fig,axs=plt.subplots(1,3,figsize=(14,4),constrained_layout=True)
    #     for i,(ax,ylabel) in enumerate(zip(axs,ylabel_lst)):
    #         ax.plot(mean_array[:,i])
    #         ax.fill_between(mean_array[:,i]-1.96*std_array[:,i],mean_array[:,i]+1.96*std_array[:,i],alpha=0.5)
    #         format_plot(ax=ax,xlabel='t (sec)',ylabel=ylabel)
    #     plt.show()

    #DONE: compute the apparent attraction coefficient without smoothing
    # np.argwhere(np.isnan(position_array[:,0,i])).flatten()
    #identify any pairs that didn't end up closer than rend
    boo_didnt_finish=~np.isnan(position_array[-1,0,:])
    pairs_didnt_finish=np.argwhere(boo_didnt_finish).flatten()
    #     if printing:
    #         print((boo_didnt_finish.shape,pairs_didnt_finish))
    set_pairs_ignore=set(pairs_didnt_finish)

    tmax_recall=0.055 #max time to recall before the value is not nan
    num_rows_recall=np.int(np.around(tmax_recall/Dt))
    if printing:
        print(num_rows_recall)

    #for each pair, identify the last row where the value is not nan
    row_end_values=zero_values.copy()+np.nan
    for i in range(num_pairs):
        if not set_pairs_ignore.issuperset({i}):
            row_end=np.int(np.argwhere(np.isnan(position_array[:,0,i]))[0,0])-1
            row_end_values[i]=row_end
    #         #this step can introduce some error
    #         if row_end<num_rows_recall:
    #             #add this i to the ignore list
    #             set_pairs_ignore.add(i)

    #DONE: add pair to set_pairs_ignore if it ends sooner than num_rows_recall

    #for each pair not to be ignored, put the last num_rows_recall before row_end_values into an array
    positions_lst=[]
    for i in range(num_pairs):
        if not set_pairs_ignore.issuperset({i}):
            row_end=np.int(row_end_values[i])
            if row_end>=num_rows_recall:
                positions=position_array[row_end-num_rows_recall:row_end,:,i]
            else:
                num_rows_rec=row_end
                num_rows_pad=num_rows_recall-row_end
                #pad the start with nans
                nan_pad=np.full([num_rows_pad,4], np.nan)
                positions=position_array[row_end-num_rows_rec:row_end,:,i]
                #pad positions at its front
                positions=np.concatenate((nan_pad,positions))

            positions_lst.append(positions)
    positions_out=np.stack(positions_lst,axis=-1)
    del position_array
    del positions_lst
    return positions_out,mean_array,std_array

In [37]:
def moving_average(a, n=3):
    ret = np.cumsum(a, dtype=float,axis=0)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [44]:
def compute_attraction_coefficient_results(positions_out,tavg1,tavg2,Dt,printing=False):
    '''
    Example Usage:
    df=compute_attraction_coefficient_results(positions_out,tavg1,tavg2,Dt)
    '''
    navg1=np.int(np.around(tavg1/(Dt*10**3)))
    navg2=np.int(np.around(tavg2/(Dt*10**3)))

    #define time points
    #i=0
    num_timepoints=positions_out.shape[0]
    t_values=np.arange(0,Dt*num_timepoints,Dt)
    #assert t_values.shape==range_values[:,i].shape
    
    #moving avg
    if tavg1>0:
        #compute the moving average of the positional trajectories
        tavg_values=moving_average(t_values,n=navg1)
        positions_out_ma=moving_average(positions_out, n=navg1)
    else:
        tavg_values=t_values
        positions_out_ma=positions_out

    #compute range # x1,y1,x2,y2
    range_values= np.sqrt((positions_out_ma[:,0,:]-positions_out_ma[:,2,:])**2 + (positions_out_ma[:,1,:]-positions_out_ma[:,3,:])**2)



    #compute savgol filter
    if navg2>0:
        if navg2<30: navg2=30
        if navg2%2==0:navg2+=1
        #DONE: test whether there is a significant difference between the savgol derivative and the finite difference derivative of the savgol filter
        savgol_kwargs = dict(window_length=navg2,polyorder=3,deriv=0,delta=1.0,axis=-1,mode='interp')
        #compute the savgol_filtered values for R
        R_savgol_values=savgol_filter(range_values,**savgol_kwargs)
        SR_values=R_savgol_values**2
    else:
        SR_values=range_values**2

    #DONE: sompute MSR_values from SR_values
    # SR_values

    if printing:
        print('nota bene: nan values are being kept in positions_out, so care needs to be taken to compute the particle averaged SR for every time')
        print(f"HINT: I need to compute the nonnan boolean index for each time slice")
    np.isnan(SR_values).any(),SR_values.shape
    # assert not np.isnan(positions_out).any()

    # the nonnan boolean index for each time slice
    MSR_values=np.zeros_like(SR_values[:,0])
    num_tbin,num_trials=SR_values.shape
    for j in range (num_tbin):
        boo=~np.isnan(SR_values[j,:])
        MSR_values[-j]=np.mean(SR_values[j,boo])

    t_thresh_values=np.arange(0.005,0.04,0.005)

    start=time.time()
    #generate apparent attraction coefficient data
    a_sr_lst=[]
    Delta_a_sr_lst=[]
    Rsq_sr_lst=[]
    # a_one_over_R_lst=[]
    # Delta_a_one_over_R_lst=[]
    # Rsq_one_over_R_lst=[]
    tdeath_values=np.max(tavg_values)-tavg_values
    for t_thresh in t_thresh_values:
        #squared range method
        boo=tdeath_values<=t_thresh
        dict_sr=compute_slope.compute_95CI_ols(tdeath_values[boo], MSR_values[boo])
        a_sr=dict_sr['m']/4
        Delta_a_sr=dict_sr['Delta_m']/4

    #     #inverse range method
    #     boo=one_over_R_values<1/R_thresh
    #     dict_one_over_R=compute_slope.compute_95CI_ols(one_over_R_values[boo], dRdt_values[boo])
    #     a_one_over_R=-1*dict_one_over_R['m']/2
    #     Delta_a_one_over_R=dict_one_over_R['Delta_m']/2

        #record results
        a_sr_lst.append(a_sr)
        Delta_a_sr_lst.append(Delta_a_sr)
        Rsq_sr_lst.append(dict_sr['Rsquared'])
    #     a_one_over_R_lst.append(a_one_over_R)
    #     Delta_a_one_over_R_lst.append(Delta_a_one_over_R)
    #     Rsq_one_over_R_lst.append(dict_one_over_R['Rsquared'])
    if printing:
        print(f"runtime computing attraction coefficients: {time.time()-start:.4f}")

    df=pd.DataFrame({
        'tavg1':tavg1,
        'tavg2':tavg2,
        'tmax':t_thresh_values,
        'a':a_sr_lst,
        'Delta_a':Delta_a_sr_lst,
        'Rsq_a':Rsq_sr_lst
    })
    return df

In [54]:
def save_control_results_for_setting(a,
                                     D,
                                     tavg_step=0.1,
                                     tavg1_max=20,
                                     tavg2_max=20,
                                     printing=False,
                                    save_folder=None):
    if save_folder==None:
        nb_dir=os.getcwd()
        save_folder = f"{nb_dir}/data/control_data/arrays"
    if not os.path.exists(save_folder):
        os.mkdir(save_folder)
    save_dir = os.path.join(save_folder, f"control_a_{a}_D_{D}.npz")

    use_cache = False
    caching = False

    if use_cache:
        arr = np.load(save_dir)
        positions_out = arr['arr_0']
    else:
        positions_out, mean_array, std_array = gener_positions_control(
            a=a, D=D, printing=True)

    if caching:
        #save array of positions to .npz
        np.savez_compressed(save_dir, positions_out)
        assert (os.path.exists(save_dir))

        if printing:
            print('saving array in:')
            print(save_dir)

    df_lst = []
    for tavg1 in np.arange(0, tavg1_max, tavg_step):
        for tavg2 in np.arange(0, tavg2_max, tavg_step):
            df = compute_attraction_coefficient_results(
                positions_out, tavg1, tavg2, Dt)
            df_lst.append(df)
    df_out = pd.concat(df_lst)
    del df_lst

    csv_dir = save_dir.replace(f'arrays/control_a_{a}_D_{D}.npz',
                               f'control_a_{a}_D_{D}.csv')
    df_out.to_csv(csv_dir, index=False)
    return csv_dir

# single simulation of diffusion in an unbounded domain
- TODO: grid search over D from 0.1 to 2 in steps of 0.1
- (GPU accelerate in cupy)
- TODO: rewrite it from scratch, developing in vectorized pairs of particles
- start 2 particles 1/np.sqrt(2) away from one another

__Schema__
1. t,x1,x2,y1,y2,pair_num

In [53]:
num_pairs=1500 #number of independent pairs of particles
a=5 #cm^2/s known attraction coefficient 
D=1 #cm^2/s known diffusion coefficient
Dt=1e-5
print_every=100000

In [55]:
# positions_out,mean_array,std_array=gener_positions_control(a=a,D=D,printing=True)

In [56]:
# df_lst=[]
# for tavg1 in np.arange(0,1.,0.1):
#     for tavg2 in np.arange(0,1.,0.1):
#         df=compute_attraction_coefficient_results(positions_out,tavg1,tavg2,Dt)
#         df_lst.append(df)
# df_out=pd.concat(df_lst)
# print(len(df_lst))
# del df_lst
# save_folder=f"{nb_dir}/data/control_data/arrays"
# if not os.path.exists(save_folder):
#     os.mkdir(save_folder)
# save_dir=os.path.join(save_folder,f"control_a_{a}_D_{D}.npz")
# csv_dir=save_dir.replace(f'arrays/control_a_{a}_D_{D}.npz',f'control_a_{a}_D_{D}.csv')
# df_out.to_csv(csv_dir,index=False)

In [57]:
# # tavg1,tavg2
# a,D

In [29]:
t_values,navg1

(array([0.000e+00, 1.000e-05, 2.000e-05, ..., 5.497e-02, 5.498e-02,
        5.499e-02]),
 0)

In [21]:
#TODO: fix the dask bag implementation

In [58]:
# beep(5)

In [59]:
task_lst=[]
for a in [5,2]:
    for D in np.arange(0,1.1,0.1)[4:]:
        task_lst.append((a,D))
        
def routine(data):
    a,D=data
    return save_control_results_for_setting(a,D)

In [60]:
testing=True
if testing:
    fn_out=save_control_results_for_setting(a,D,
                                    tavg_step=0.1,
                                    tavg1_max=1,
                                    tavg2_max=1,printing=True)
    df=pd.read_csv(fn_out)
    print(df.head())

running simulation for 1000000 steps...
Simulation 10% completed: R=0.1420+/-0.2783, diffusion/attraction is 15.27%
Simulation 20% completed: R=0.1035+/-0.2029, diffusion/attraction is 14.85%
Simulation 30% completed: R=0.0866+/-0.1697, diffusion/attraction is 14.65%
Simulation 40% completed: R=0.0595+/-0.1166, diffusion/attraction is 14.68%
Simulation 50% completed: R=0.0578+/-0.1132, diffusion/attraction is 14.78%
Simulation 60% completed: R=0.0530+/-0.1038, diffusion/attraction is 14.55%
Simulation 70% completed: R=0.0490+/-0.0960, diffusion/attraction is 14.49%
Simulation 80% completed: R=0.0573+/-0.1123, diffusion/attraction is 14.62%
Simulation 90% completed: R=0.0564+/-0.1105, diffusion/attraction is 14.41%
Simulation 100% completed: R=0.0561+/-0.1099, diffusion/attraction is 14.86%
simulation complete!
Total run time: 313.5142 seconds
the number of particles that didn't finished is 10 out of 1500.
((1500,), array([ 240,  323,  356,  844,  921, 1122, 1138, 1219, 1221, 1433]))
55

In [18]:
print(f"note: approximately 45 GB of RAM is needed per worker")
npartitions=1
bag = db.from_sequence(task_lst, npartitions=npartitions).map(routine)
start = time.time()
retval_lst = list(bag)
print(f"the run time for filtering files was {(time.time()-start)/60:.2f} minutes.")
# return retval_lst
print(len(retval_lst))

note: approximately 45 GB of RAM is needed per worker
running simulation for 1000000 steps...
Simulation 10% completed: R=0.0007+/-0.0013, diffusion/attraction is 3.57%
Simulation 20% completed: R=0.0007+/-0.0013, diffusion/attraction is 3.59%
Simulation 30% completed: R=0.0007+/-0.0013, diffusion/attraction is 3.56%
Simulation 40% completed: R=0.0007+/-0.0013, diffusion/attraction is 3.59%
Simulation 50% completed: R=0.0007+/-0.0013, diffusion/attraction is 3.60%
Simulation 60% completed: R=0.0007+/-0.0013, diffusion/attraction is 3.58%
Simulation 70% completed: R=0.0007+/-0.0013, diffusion/attraction is 3.66%
Simulation 80% completed: R=0.0007+/-0.0013, diffusion/attraction is 3.62%
Simulation 90% completed: R=0.0007+/-0.0013, diffusion/attraction is 3.65%
Simulation 100% completed: R=0.0007+/-0.0013, diffusion/attraction is 3.57%
simulation complete!
Total run time: 285.1986 seconds
the number of particles that didn't finished is 0 out of 1500.
((1500,), array([], dtype=int64))
5500

ValueError: operands could not be broadcast together with shapes (5500,) (0,) 

# visualization

In [ ]:
for i in range(100):#[5,7,100]:
    plt.plot(t_values,range_values[::-1,i],c='gray',lw=2,alpha=0.5)
format_plot(ax=plt.gca(),xlabel='t (sec)',ylabel=r'MSR (cm$^2$)')
plt.show()

In [ ]:
#TODO: make a simple plot of MSR vs tf-t
# for i in range(100):#[5,7,100]:
#     plt.plot(t_values,range_values[::-1,i]**2,c='gray',lw=2,alpha=0.5)
plt.plot(t_values[1:],MSR_values[1:]**2,c='C0',lw=2,alpha=0.5)
format_plot(ax=plt.gca(),xlabel=r'$t_f-t$ (sec)',ylabel=r'MSR (cm$^2$)')
plt.show()

In [ ]:
#TODO: visualize a_SR versus tmax
a_SR_lst

In [ ]:
#TODO: try to understand what the quadratic MSR versus t means

In [ ]:
#TODO: wrap map from tavg1,tavg2 to a_SR et al

In [ ]:
#TODO: dev grid search over tavg1,tavg2 to a dataframe of a_SR et al

In [ ]:
#TODO: save ^that dataframe as csv

In [ ]:
#(supposing the tavg1,tavg2 grid search is reasonably fast <1 hour),
#TODO: run a dask bag over D=0.,...,1. in min(0.1,x), where x = the minimum stepsize that should finish by the time I wakeup tomorrow
#nota bene: without too much difficulty, I could use 50 cores from ssh-miller and use ~5X smaller x...